In [4]:
import pandas as pd


In [5]:
confirmed_df = pd.read_csv('sample_data/time_series_covid19_confirmed_global.csv')
death_df = pd.read_csv('sample_data/time_series_covid19_deaths_global.csv')
confirmed_df.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/14/22,4/15/22,4/16/22,4/17/22,4/18/22,4/19/22,4/20/22,4/21/22,4/22/22,4/23/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,178352,178373,178387,178418,178457,178513,178574,178611,178638,178648
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,274376,274429,274462,274504,274520,274535,274606,274606,274737,274791


Use pandas to create a dataframe that aggregates and sums both confirmed cases and deaths on a global level

#Transforming the Date format from 1/2/20 to YYYY-MM-DD (2020-01-02)
 

In [6]:
total_confirmed= confirmed_df.iloc[:, 4:].sum(axis=0)
total_death= death_df.iloc[:, 4:].sum(axis=0)

frame = { 'Covid Confirmed': total_confirmed, 'Covid Death': total_death }
aggregated_df = pd.DataFrame(frame )

as_list = aggregated_df.index.tolist()
new_list = []
for i in as_list:
    nums = i.split('/')
    month_leading_zero='0'
    if(int(nums[0]) > 9): month_leading_zero = ''

    day_leading_zero='0'
    if(int(nums[1]) > 9): day_leading_zero = ''

    new_list.append('20'+nums[2]+'-'+month_leading_zero+''+nums[0]+'-'+day_leading_zero+nums[1])

aggregated_df.index = new_list
aggregated_df = aggregated_df.transpose()
aggregated_df.head(5)

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2022-04-14,2022-04-15,2022-04-16,2022-04-17,2022-04-18,2022-04-19,2022-04-20,2022-04-21,2022-04-22,2022-04-23
Covid Confirmed,557,657,944,1437,2120,2929,5580,6169,8237,9927,...,502932802,503645731,504198041,504619899,505089291,506082451,507046645,507947338,508708884,509196481
Covid Death,17,18,26,42,56,82,131,133,171,213,...,6193401,6195670,6197182,6198487,6199914,6203317,6207815,6211981,6215610,6217046


Alpha Vantage! API key: 91MNBSNGPV50MFCO. 

In [7]:
!pip install alpha_vantage

In [8]:
from alpha_vantage.timeseries import TimeSeries
# ts = TimeSeries(key='91MNBSNGPV50MFCO')
# 
ts = TimeSeries(key='ED7TWFSOZVK81POS')
# Get json object with the intraday data and another with  the call's metadata


In [9]:
# series = pd.read_json(data, typ='series', orient='records')
# print(series)
# Google - Googl - American
data1, meta_data1 = ts.get_daily(symbol='GOOGL',outputsize='full')
# Royal Bank Canada Aka RBC - TSE - Canadian
data2, meta_data2 = ts.get_daily(symbol='TSE',outputsize='full')
# AirBnb - ABNB - Travel Sector
data3, meta_data3 = ts.get_daily(symbol='ABNB',outputsize='full')
# Annaly Capital Management Inc. - NLY - Real estate
data4, meta_data4 = ts.get_daily(symbol='NLY',outputsize='full')
# Gold - AU - Metal
data5, meta_data5 = ts.get_daily(symbol='AU',outputsize='full')

#Filling the NAN values of stocks with previous date values.

In [12]:
google_df = pd.DataFrame(data1)
google_df = google_df.ffill()

rbc_df = pd.DataFrame(data2)
rbc_df = rbc_df.ffill()

airbnb_df =pd.DataFrame(data3)
airbnb_df = airbnb_df.ffill()

nly_df = pd.DataFrame(data4)
nly_df = nly_df.ffill()

gold_df = pd.DataFrame(data5)
gold_df = gold_df.ffill()
gold_df.head(5)

,2022-04-25,2022-04-22,2022-04-21,2022-04-20,2022-04-19,2022-04-18,2022-04-14,2022-04-13,2022-04-12,2022-04-11,...,1999-11-12,1999-11-11,1999-11-10,1999-11-09,1999-11-08,1999-11-05,1999-11-04,1999-11-03,1999-11-02,1999-11-01
1. open,20.2100,21.3800,22.3000,22.3200,23.9000,24.1100,23.5100,23.8000,23.9300,24.4000,...,28.5000,28.9400,28.6300,28.1300,28.1900,28.8100,28.0600,27.3800,27.0600,27.5600
2. high,20.7100,21.7400,22.3000,22.7700,23.9900,24.9700,23.7300,24.1450,24.2300,24.4100,...,28.8800,28.9400,29.1900,28.9400,28.4400,28.9400,28.3800,28.0000,27.3100,27.5600
3. low,19.9200,20.9000,21.4700,22.0700,22.3500,23.9400,23.1563,23.5700,23.2200,22.8450,...,28.3800,28.6900,28.6300,27.6300,27.9400,28.5000,28.0000,27.3100,27.0000,26.8800
4. close,20.4200,21.1700,21.7700,22.6900,22.5700,24.3000,23.6800,23.8900,23.5800,23.1200,...,28.6300,28.7500,29.1300,28.6300,28.3800,28.6300,28.2500,27.5600,27.3100,27.0000
5. volume,2901349,2607946,2659717,2177515,3747691,2725169,2515974,1752183,2644686,3360571,...,121700,155600,426400,267600,224600,328100,133300,190600,420700,247600


# Combining the data in order to align data according to time series

In [ ]:
# print(aggregated_df.head(2))
def delete(transposed_df):
  return transposed_df.drop(columns=['1. open','2. high','3. low','4. close','5. volume'])
# print(google_df.loc('2. high'))
transposed_df= aggregated_df.transpose()
# American
transposed_df = transposed_df.append(google_df.transpose())
transposed_df['American high'] = transposed_df['2. high']
transposed_df['American low'] = transposed_df['3. low']
transposed_df = delete(transposed_df)
# Canadian
transposed_df = transposed_df.append(rbc_df.transpose())
transposed_df['Canadian high'] = transposed_df['2. high']
transposed_df['Canadian low'] = transposed_df['3. low']
transposed_df = delete(transposed_df)
#  Travel
transposed_df = transposed_df.append(airbnb_df.transpose())
transposed_df['Travel high'] = transposed_df['2. high']
transposed_df['Travel low'] = transposed_df['3. low']
transposed_df = delete(transposed_df)
#  Real Estate
transposed_df = transposed_df.append(nly_df.transpose())
transposed_df['Real Estate high'] = transposed_df['2. high']
transposed_df['Real Estate  low'] = transposed_df['3. low']
transposed_df = delete(transposed_df)
# Metal gold_df
transposed_df = transposed_df.append(gold_df.transpose())
transposed_df['Metal high'] = transposed_df['2. high']
transposed_df['Metal low'] = transposed_df['3. low']
# 
transposed_df = delete(transposed_df)
transposed_df = transposed_df.sort_index()
transposed_df=transposed_df.ffill(axis=0)

# Cleansing the data for COVID NaN values 
transposed_df = transposed_df.dropna()

#
transposed_df['American high'] = transposed_df['American high'].astype(float)
transposed_df['Canadian high'] = transposed_df['Canadian high'].astype(float) 
transposed_df['Travel high'] = transposed_df['Travel high'].astype(float) 
transposed_df['Real Estate high'] = transposed_df['Real Estate high'].astype(float) 
transposed_df['Metal high'] = transposed_df['Metal high'].astype(float) 

transposed_df.head(10)